# Transformer model architecture
![](https://github.com/PytLab/transformer-from-scratch/blob/master/images/transformer_architecture.png?raw=1)

# Transformer implementation

In [77]:
import torch

## Embed

### Embedding

In [78]:
class Embedder(torch.nn.Module):
    
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        # [123, 0, 23, 5] -> [[..512..], [...512...], ...]
        return self.embed(x)

### Positional encoding

$$ PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}}) $$

$$ PE_{(pos, 2i + 1)} = cos(pos/10000^{2i/d_{model}}) $$

In [79]:
import math

class PositionalEncoder(torch.nn.Module):
    
    def __init__(self, d_model, max_seq_len=80):
        super().__init__()
        self.d_model = d_model
        
        # create constant positional encoding matrix
        pe_matrix = torch.zeros(max_seq_len, d_model)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe_matrix[pos, i] = math.sin(pos/10000**(2*i/d_model))
                pe_matrix[pos, i+1] = math.cos(pos/10000**(2*i/d_model))
        pe_matrix = pe_matrix.unsqueeze(0)     # Add one dimension for batch size
        self.register_buffer('pe', pe_matrix)  # Register as persistent buffer
        
    def forward(self, x):
        # x is a sentence after embedding with dim (batch, number of words, vector dimension)
        seq_len = x.size()[1]
        x = x + self.pe[:, :seq_len]
        return x

## Model layers

### Scaled Dot-Product Attention layer

![](https://github.com/PytLab/transformer-from-scratch/blob/master/images/scaled_dot_product_attention.png?raw=1)

12*512

[512..] x M = k_0
[...512] x M = k_1

In [80]:
import math
import torch.nn.functional as F

# Given Query, Key, Value, calculate the final weighted value
def scaled_dot_product_attention(q, k, v, mask=None, dropout=None):
    # Shape of q and k are the same, both are (batch_size, seq_len, d_k)
    # Shape of v is (batch_size, seq_len, d_v)
    attention_scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(q.shape[-1])  # size (batch_size, seq_len, seq_len)
    
    # Apply mask to scores
    # <pad>
    if mask is not None:
        attention_scores = attention_scores.masked_fill(mask == 0, value=-1e9)
        
    # Softmax along the last dimension
    attention_weights = F.softmax(attention_scores, dim=-1)
    
    if dropout is not None:
        attention_weights = dropout(attention_weights)
        
    output = torch.matmul(attention_weights, v)
    return output

### Multi-Head Attention layer

![](https://github.com/PytLab/transformer-from-scratch/blob/master/images/multi_head_attention.png?raw=1)

In [81]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        super().__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = self.d_v = d_model//n_heads
        
        # self attention linear layers
        # Linear layers for q, k, v vectors generation in different heads
        self.q_linear_layers = []
        self.k_linear_layers = []
        self.v_linear_layers = []
        for i in range(n_heads):
            self.q_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.k_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.v_linear_layers.append(torch.nn.Linear(d_model, self.d_v))
        
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(n_heads*self.d_v, d_model)
        
    def forward(self, q, k, v, mask=None):
        multi_head_attention_outputs = []
        for q_linear, k_linear, v_linear in zip(self.q_linear_layers,
                                                self.k_linear_layers,
                                                self.v_linear_layers):
            new_q = q_linear(q)  # size: (batch_size, seq_len, d_k)
            new_k = k_linear(k)  # size: (batch_size, seq_len, d_k)
            new_v = v_linear(v)  # size: (batch_size, seq_len, d_v)
            
            # Scaled Dot-Product attention
            head_v = scaled_dot_product_attention(new_q, new_k, new_v, mask, self.dropout)  # (batch_size, seq_len, d_v)
            multi_head_attention_outputs.append(head_v)
            
        # Concat
        #import pdb; pdb.set_trace()
        concat = torch.cat(multi_head_attention_outputs, -1)  # (batch_size, seq_len, n_heads*d_v)
        
        # Linear layer to recover to original shap
        output = self.out(concat)  # (batch_size, seq_len, d_model)
        
        return output

### Feed Forward layer

In [82]:
class FeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        
        self.linear_1 = torch.nn.Linear(d_model, d_ff)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear_2 = torch.nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

### Layer Normalization layer

#### Normalization

$$\mu = \frac{1}{m} \sum_{i=1}^{m}x_i$$

$$
\sigma^{2} = \frac{1}{m} \sum^{m}_{i=1}(x_{i} - \mu)^2
$$

$$
\hat{Z}_i = \frac{x_i - \mu_i}{\sqrt{\sigma^{2}_{i} + \epsilon}}
$$

#### Add two learnable parameters

$$
\tilde{Z}_i = \alpha_i * \hat{Z}_i + \beta_i
$$

In [83]:
class LayerNorm(torch.nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.d_model = d_model
        self.alpha = torch.nn.Parameter(torch.ones(self.d_model))
        self.beta = torch.nn.Parameter(torch.zeros(self.d_model))
        self.eps = eps
        
    def forward(self, x):
        # x size: (batch_size, seq_len, d_model)
        x_hat = (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps)
        x_tilde = self.alpha*x_hat + self.beta
        return x_tilde

## Encoder & Decoder layers

### Encoder layer

An encoder layer contains a multi-head attention layer and feed forward layer

![](https://github.com/PytLab/transformer-from-scratch/blob/master/images/encoder.png?raw=1)

In [84]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.multi_head_attention = MultiHeadAttention(n_heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        
    def forward(self, x, mask):
        #import pdb; pdb.set_trace()
        x = x + self.dropout_1(self.multi_head_attention(x, x, x, mask))
        x = self.norm_1(x)
        
        x = x + self.dropout_2(self.feed_forward(x))
        x = self.norm_2(x)
        return x

### Decoder layer

An decoder layer contains two multi-head attention layers and one feed forward layer

![](https://github.com/PytLab/transformer-from-scratch/blob/master/images/decoder.png?raw=1)

In [85]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.norm_3 = LayerNorm(d_model)
        
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        self.dropout_3 = torch.nn.Dropout(dropout)
        
        self.multi_head_attention_1 = MultiHeadAttention(n_heads, d_model)
        self.multi_head_attention_2 = MultiHeadAttention(n_heads, d_model)
        
        self.feed_forward = FeedForward(d_model)
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        x = self.dropout_1(self.multi_head_attention_1(x, x, x, trg_mask))
        x = x + self.norm_1(x)
        
        x = self.dropout_2(self.multi_head_attention_2(x, encoder_output, encoder_output, src_mask))
        x = x + self.norm_2(x)
        
        x = self.dropout_3(self.feed_forward(x))
        x = x + self.norm_3(x)
        
        return x

In [86]:
import copy

def clone_layer(module, N):
    return torch.nn.ModuleList([copy.deepcopy(module) for i in range(N)])

## Encoder & Decoder

In [87]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.encoder_layers = clone_layer(EncoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for encoder in self.encoder_layers:
            x = encoder(x, mask)
        return self.norm(x)

In [88]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.decoder_layers = clone_layer(DecoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, trg, encoder_output, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for decoder in self.decoder_layers:
            x = decoder(x, encoder_output, src_mask, trg_mask)
        return self.norm(x)

## Transformer

In [89]:
class Transformer(torch.nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, N, n_heads):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads)
        self.decoder = Decoder(trg_vocab_size, d_model, N, n_heads)
        self.linear = torch.nn.Linear(d_model, trg_vocab_size)
        
    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(src, src_mask)
        decoder_output = self.decoder(trg, encoder_output, src_mask, trg_mask)
        output = self.linear(decoder_output)
        return output

# Data processing

In [90]:
import spacy

!pip install -U torchtext==0.10.0
from torchtext.legacy import data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
nlp = spacy.load('en_core_web_sm')

tk = nlp.tokenizer("I love you so-much!!!")

tokenizer = lambda sentence: [tok.text for tok in nlp.tokenizer(sentence) if tok.text != " "]

print(tokenizer("Allo la terre"))

['Allo', 'la', 'terre']


In [122]:
SRC = data.Field(lower=True, tokenize=tokenizer)
TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')

In [93]:
src_data = open('data/english.txt', 'r')

In [94]:
trg_data = open('data/french.txt', 'r')

In [95]:
raw_data = {'src': [line for line in src_data], 'trg': [line for line in trg_data]}

In [96]:
import pandas as pd

In [97]:
df = pd.DataFrame(raw_data, columns=['src', 'trg'])

In [98]:
df

,src,trg
0,Go.\n,Va !\n
1,Run!\n,Cours !\n
2,Run!\n,Courez !\n
3,Fire!\n,Au feu !\n
4,Help!\n,À l'aide !\n
...,...,...
154878,"""Top-down economics never works,"" said Obama. ...","« L'économie en partant du haut vers le bas, ç..."
154879,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
154880,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
154881,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [99]:
df.to_csv('en_to_fr.csv', index=False)

In [100]:
ls

data/  en_to_fr.csv  sample_data/


In [123]:
data_fields = [('src', SRC), ('trg', TRG)]

In [124]:
train_set = data.TabularDataset('./en_to_fr.csv', format='csv', fields=data_fields)

In [125]:
print(train_set)

In [126]:
SRC.build_vocab(train_set)

In [179]:
len(SRC.vocab)
# print(SRC.vocab.itos[2])

.


In [128]:
TRG.build_vocab(train_set)

In [129]:
len(TRG.vocab)

26341

In [130]:
train_set

# Train transformer

In [131]:
# set some parameters
d_model = 512
n_heads = 8
N = 6
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)

In [132]:
model = Transformer(src_vocab_size, trg_vocab_size, d_model, N, n_heads)

In [133]:
for p in model.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform(p)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [134]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [184]:
train_iter = data.Iterator(train_set, batch_size=1, sort_key=lambda x: (len(x.src), len(x.trg)), shuffle=True, train=True)

In [185]:
import numpy as np

def create_mask(src_input, trg_input):
    # Source input mask
    pad = SRC.vocab.stoi['<pad>']
    src_mask = (src_input != pad).unsqueeze(1)
    
    # Target input mask
    trg_mask = (trg_input != pad).unsqueeze(1)
    
    seq_len = trg_input.size(1)
    nopeak_mask = np.tril(np.ones((1, seq_len, seq_len)), k=0).astype('uint8')
    nopeak_mask = torch.from_numpy(nopeak_mask) != 0
    trg_mask = trg_mask & nopeak_mask
    
    return src_mask, trg_mask

In [192]:
import time

def train_model(n_epochs, output_interval=100):
    model.train()
    start = time.time()
    
    for epoch in range(n_epochs):
        
        total_loss = 0
        for i, batch in enumerate(train_iter):
            
            
            src_input = batch.src.transpose(0, 1)  # size (batch_size, seq_len)
            trg = batch.trg.transpose(0, 1)  # size (batch_size, seq_len)

            
            
            trg_input = trg[:, :-1]
            ys = trg[:, 1:].contiguous().view(-1)

            print("src input:")
            print(src_input)
            print()

            print("target:")
            print(trg)
            print()

            print("target input:")
            print(trg_input)
            print()

            print("ys:")
            print(ys)
            print()
            # create src & trg masks
            src_mask, trg_mask = create_mask(src_input, trg_input)
            preds = model(src_input, trg_input, src_mask, trg_mask)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=1)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data

            if (i + 1) % output_interval == 0:
                avg_loss = total_loss/output_interval
                print('time = {}, epoch = {}, iter = {}, loss = {}'.format((time.time() - start)/60,
                                                                           epoch + 1,
                                                                           i + 1,
                                                                           avg_loss))
                total_loss = 0
                start = time.time()

In [194]:
train_model(3, output_interval=1)

src input:
tensor([[ 31,  16,   8, 868, 362,   2]])

target:
tensor([[   2,   34,   27,  253, 2162,    4,    3]])

target input:
tensor([[   2,   34,   27,  253, 2162,    4]])

ys:
tensor([  34,   27,  253, 2162,    4,    3])

time = 0.02362548510233561, epoch = 1, iter = 1, loss = 8.015908241271973
src input:
tensor([[318, 153,  11, 141,   7]])

target:
tensor([[  2, 156, 164,  19,  14, 457,  33,   8,   3]])

target input:
tensor([[  2, 156, 164,  19,  14, 457,  33,   8]])

ys:
tensor([156, 164,  19,  14, 457,  33,   8,   3])

time = 0.015025500456492107, epoch = 1, iter = 2, loss = 8.71261978149414
src input:
tensor([[ 30,   4,  86,   8, 758,  56,   7]])

target:
tensor([[  2,  70,   7,  17, 145,  23, 699,  81,  33,   8,   3]])

target input:
tensor([[  2,  70,   7,  17, 145,  23, 699,  81,  33,   8]])

ys:
tensor([ 70,   7,  17, 145,  23, 699,  81,  33,   8,   3])

time = 0.01535019079844157, epoch = 1, iter = 3, loss = 8.65533447265625
src input:
tensor([[ 12,  16,  53,  54,   3,  

KeyboardInterrupt: ignored